In [ ]:
import torch
from unsloth import FastLanguageModel

if torch.cuda.is_available():
    print("GPU is available!")
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs: {num_gpus}")
    current_gpu_name = torch.cuda.get_device_name(torch.cuda.current_device())
    print(f"Current GPU: {current_gpu_name}")
else:
    print("GPU not available. Training will be very slow.")

max_seq_length = 2048
dtype = None

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="XiaomiMiMo/MiMo-7B-RL-0530",
    max_seq_length=max_seq_length,
    dtype=dtype,
)

In [ ]:
import json
from datasets import load_dataset

dataset = load_dataset("json", data_files="data/data.json")

In [ ]:
from datasets import Dataset

def formatting_prompts_func(examples):
    chats = examples["messages"]
    texts = [tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False) for chat in chats]
    return {"text": texts}

formatted_dataset = dataset.map(formatting_prompts_func, batched=False)

In [ ]:
from peft import LoraConfig
from trl import SFTTrainer
from transformers import TrainingArguments

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsupervised",
    random_state=42,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False, 
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs",
    ),
)

In [ ]:
training_stats = trainer.train()

In [ ]:
model.save_pretrained("finetuned_film_model")